In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GitHub" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect


NOTE: User is responsible for checking the content of datasets and the applicable licenses and determining if suitable for the intended use.
"""
# If you're using Google Colab and not running locally, run this cell.
import os

# Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'main'
!python -m pip install "nemo_toolkit[asr] @ git+https://github.com/NVIDIA/NeMo.git@$BRANCH"

# Multi Task Adaptation with Adapters


In earlier tutorials, we utilized a specific model for one task - for example, an ASR model (CTC, RNN-T etc) for the singular task of Speech Recognition. This is very useful if we want to specialize one task per model, but it can be expensive to deploy a fleet of models for each task, and learn routers to pass user tasks to correct models.

We now support Multi Task models in NeMo, such that a single model can perform multiple tasks such as speech recognition, speech translation, voice activity detection, and more in the future. With one model supporting multiple tasks, we can simplify the task of deploying models and also hope to leverage individual tasks to improve each other (for example: you do need strong speech recognition first before you start doing translation).

---

Multi Task (Canary) models are highly capable large neural networks capable of things like speech recognition, X to English and English to X translation and able to select whether to transcribe speech with punctuation and capitalization. These huge models are trained on several thousand hours of speech and text data, making it challenging to adapt to new datasets.

In the previous tutorial for [ASR Adapters](https://github.com/NVIDIA/NeMo/blob/main/tutorials/asr/asr_adapters/ASR_with_Adapters.ipynb), we used small adapter modules to tune a large ASR model on a small amount of data. In this tutorial, we will adapt a [Nvidia Canary](https://huggingface.co/nvidia/canary-1b) model onto a small amount of speech data for both Automatic Speech Recognition (ASR) and Automatic Speech Translation (AST).

In this tutorial, we will also demonstrate a simple way of creating custom Data Modules from PyTorch Lightning to design custom datasets and data loaders for the highly flexible Multi Task Models in NeMo ASR. This offers users more flexibility in designing new tasks, and finetuning the models on small amounts of data.

----

First, lets instantiate the [Canary](https://huggingface.co/nvidia/canary-1b) model

In [1]:
import os
import json

import nemo.collections.asr as nemo_asr

In [2]:
model = nemo_asr.models.ASRModel.from_pretrained("nvidia/canary-1b")

[NeMo I 2024-11-06 05:36:04 mixins:200] _setup_tokenizer: detected an aggregate tokenizer
[NeMo I 2024-11-06 05:36:04 mixins:339] Tokenizer SentencePieceTokenizer initialized with 32 tokens
[NeMo I 2024-11-06 05:36:04 mixins:339] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2024-11-06 05:36:04 mixins:339] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2024-11-06 05:36:04 mixins:339] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2024-11-06 05:36:04 mixins:339] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2024-11-06 05:36:04 aggregate_tokenizer:73] Aggregate vocab size: 4128


[NeMo W 2024-11-06 05:36:04 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    batch_size: null
    num_workers: 8
    use_lhotse: true
    max_duration: 40
    pin_memory: true
    use_bucketing: false
    bucket_duration_bins: null
    num_buckets: 1
    text_field: answer
    lang_field: target_lang
    batch_duration: 360
    quadratic_duration: 15
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    
[NeMo W 2024-11-06 05:36:04 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 

[NeMo I 2024-11-06 05:36:04 features:305] PADDING: 0
[NeMo I 2024-11-06 05:36:12 save_restore_connector:275] Model EncDecMultiTaskModel was successfully restored from /home/pzelasko/.cache/huggingface/hub/models--nvidia--canary-1b/snapshots/dd32c0c709e2bfc79f583e16b9df4b3a160f7e86/canary-1b.nemo.


# Enable Adapter Support in Model

New in NeMo 2.0, we now have a simple utility function to convert the model into one that supports adapters, called `replace_adapter_compatible_modules()`.

This will go through the full model and check modules if they support adapters, and then enable that ability. Once used, you can freely use adapter methods.

In [3]:
model.replace_adapter_compatible_modules()

[NeMo I 2024-11-06 05:36:12 adapter_mixins:169] Swapping class nemo.collections.asr.modules.conformer_encoder.ConformerEncoder with adapter compatible class: nemo.collections.asr.modules.conformer_encoder.ConformerEncoderAdapter
[NeMo I 2024-11-06 05:36:12 adapter_mixins:169] Swapping class nemo.collections.asr.modules.transformer.transformer.TransformerDecoderNM with adapter compatible class: nemo.collections.asr.modules.transformer.transformer.TransformerDecoderNMAdapter
[NeMo I 2024-11-06 05:36:12 adapter_mixins:169] Swapping class nemo.collections.asr.modules.transformer.transformer_decoders.TransformerDecoder with adapter compatible class: nemo.collections.asr.modules.transformer.transformer_decoders.TransformerDecoderAdapter


## Check Which Targets Are Supported For This Model

Now that the model has enabled adapter support, lets take a look at which of its modules support adapter modules to be attached to them.

**Note**
Below, you might see an adapter module with no name `''` - this corresponds to the "default" model target if the target isn't specified. Users can chose to simply skip the module name when adding an adapter, and the model will by default add adapters to the encoder module.

In [4]:
model.adapter_module_names

['', 'encoder', 'transf_encoder', 'transf_decoder']

## Prepare the Adapter

Now that we know which modules are supported, lets create a simple adapter module for the encoder and decoder modules.

In [5]:
from nemo.collections.common.parts import LinearAdapterConfig

In [6]:
input_dim = model.cfg.encoder.d_model
adapter_dim = 8

In [7]:
enc_adapter_cfg = LinearAdapterConfig(in_features=input_dim, dim=adapter_dim)
dec_adapter_cfg = LinearAdapterConfig(in_features=input_dim, dim=adapter_dim)

## Add Adapter Modules

Now that we have the adapter configs prepared, lets add them to the model !

We provide the target module by using `target:adapter_name` when calling `add_adapter()` - this tells the model to setup an adapter called `adapter_name` to the module denoted by `target` with the config `cfg`.

In [8]:
model.add_adapter(name="encoder:enc", cfg=enc_adapter_cfg)
model.add_adapter(name="transf_decoder:dec", cfg=dec_adapter_cfg)

print("Added adapters!")

Added adapters!


## Freeze Original Module Parameters and Unfreeze Adapter Weights Only

When tuning adapters, we usually freeze the entire base model and only tune the adapters. This prevents the need for large amounts of data, preserves a lot of memory (since the full model doesnt need backward pass, only the adapters) and makes it easier to adapt huge models.

In [9]:
model.freeze()
model.unfreeze_enabled_adapters()

[NeMo I 2024-11-06 05:36:13 adapter_mixins:465] Froze module encoder.layers.0.conv.batch_norm: BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-11-06 05:36:13 adapter_mixins:465] Froze module encoder.layers.1.conv.batch_norm: BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-11-06 05:36:13 adapter_mixins:465] Froze module encoder.layers.2.conv.batch_norm: BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-11-06 05:36:13 adapter_mixins:465] Froze module encoder.layers.3.conv.batch_norm: BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-11-06 05:36:13 adapter_mixins:465] Froze module encoder.layers.4.conv.batch_norm: BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-11-06 05:36:13 adapter_mixins:465] Froze module encoder.layers.5.conv.batch_norm: BatchNorm1d(102

----

Lets make sure that the number of trainable parameters is a lot smaller (< 1 M) than the total number of params (1 B).

In [10]:
model.summarize()

  | Name                 | Type                              | Params | Mode
----------------------------------------------------------------------------------
0 | preprocessor         | AudioToMelSpectrogramPreprocessor | 0      | eval
1 | encoder              | ConformerEncoderAdapter           | 609 M  | eval
2 | encoder_decoder_proj | Identity                          | 0      | eval
3 | transf_decoder       | TransformerDecoderNMAdapter       | 407 M  | eval
4 | log_softmax          | TokenClassifier                   | 4.2 M  | eval
5 | loss                 | SmoothedCrossEntropyLoss          | 0      | eval
6 | spec_augmentation    | SpectrogramAugmentation           | 0      | eval
7 | val_loss             | GlobalAverageLossMetric           | 0      | eval
8 | wer                  | WER                               | 0      | eval
9 | bleu                 | BLEU                              | 0      | eval
----------------------------------------------------------------------

## Check Enabled Adapters

Here, we check that the adapters that we named above (`enc` and `dec`) are both setup and enabled.

In [11]:
model.get_enabled_adapters()

['dec', 'enc']

# Customizing Multi Task Models

In the following section, we will take a deeper look into what are the components that compose a Multi Task Model and how users can override each of these parts to create their own customizable multi task models.

---

In this tutorial, we will only see the internal components such as the prompt format and dataset construction, but not change them.

In a following tutorial, we will show how to add an additional task to a pre-trained Multi Task Model using a pre-trained model as a starting point.

# Prompt Handling for Multi Task Models
Nvidia Canary is our first model that is a Multi Task Model.

Multi Task models utilize a prompt format, similar to those used in Large Language Models, in order to denote to the model which task is to be performed, which langauge is being spoken and what language should the output transcript be in, whether to provide punctuation and capitalization or not, and so much more in the future !

Lets take a look at the model's `prompt` for the Canary model that we have created -

In [12]:
model.prompt_format

'canary'

----

This gives us the prompt format functions name, which we will see below points to a prompt format function that reads in manifest items and maps it to the template.

## Reuse / Register a Prompt Format Function

When we print `model.prompt_format` it writes `canary` which is one of the registered prompt templates available in NeMo ASR.
For simplicity's sake, we will continue to use the same prompt format for this tutorial. However, we enable users to define their own prompt formats and register them as needed.

Let's see what the `canary` prompt format looks like:

In [13]:
from nemo.collections.common.prompts.fn import get_prompt_format_fn, registered_prompt_format_fn

In [14]:
canary_prompt_format_fn = get_prompt_format_fn("canary")
canary_prompt_format_fn?

Signature:
canary_prompt_format_fn(
    cuts: lhotse.cut.set.CutSet,
    tokenizer: nemo.collections.common.tokenizers.tokenizer_spec.TokenizerSpec,
) -> tuple[list[torch.Tensor], list[torch.Tensor], list[torch.Tensor]]
Docstring:
Prepend and append control tokens to the token sequence as per Canary format.

We use the following special tokens:
* <|startoftranscript|>
* <|transcribe|>
* <|translate|>
* <|nopnc|>
* <|pnc|>
* <|endoftext|>
* <|LANG|> - for each supported language.
* <|nospeech|>

The prompt format syntax is as follows:

    <|startoftranscript|> [ <|nospeech|> | <|LANG|> [ <|transcribe|> | <|translate|> ] <|LANG|> [ <|pnc|> | <|nopnc|> ] TEXT <|endoftext|> ]

Where expression ``[ a | b ]`` denotes expression ``a`` or expression ``b``, and can be nested.
Note that ``<|LANG|>`` appears twice: the first occurrence is for the "source" language
(i.e., spoken language in the recording) and the second occurrence is for the "target" language
(i.e., the language in which we are g

### Registering a New Prompt Format Function

Just to show that this is user-configurable, we show how to register a dummy prompt format below:

In [15]:
@registered_prompt_format_fn
def canary2(cuts, tokenizer, inference: bool):
    """ Users can implement this as needed """
    raise NotImplementedError()

print("Registered prompt")

Registered prompt


In [16]:
temp = get_prompt_format_fn('canary2')
temp.__name__

'canary2'

## Create / Reuse a Prompt Format

Canary Multi Task Model comes with a pre-defined prompt template, so we need to provide it data in a format that can be handled by that prompt format class.

A `PromptFormatter` is a special class that defines the dialog template of the order of turns that occur in a model's prompt. For example, in Language Models, we normally may begin with either a `System` or `User` turn, followed by an `Assistant` turn which produces an output from the model. Similarly in Multi Task models, we enable support for such a usage pattern.

Do note: Current generation of Canary models are not trained to operate on multi turn conversations, however future variants of Multi Task models may support such usage.

In [17]:
# Let's review the actual prompt formatter clas docs
model.prompt?

Type:            CanaryPromptFormatter
String form:     <nemo.collections.common.prompts.canary.CanaryPromptFormatter object at 0x7f89e7d80fd0>
File:            ~/miniconda3/envs/nemo2/lib/python3.10/site-packages/nemo/collections/common/prompts/canary.py
Docstring:       <no docstring>
Class docstring:
:class:`~nemo.collections.common.prompts.formatter.PromptFormatter` is intended to simplify
working with various prompt format templates and encoding them into token ID tensors.

It assumes a dialog-like structure, which is a list of turns, with each turn assigned to a role.
Sub-classes of PromptFormatter define turn templates for each role under TEMPLATE class attribute.
Each template may define some constant parts (e.g. begin-of-turn or end-of-turn tokens, whitespaces, etc.)
and variable parts which we call "slots", that will be provided by the user during training or inference.

A role is typically "user" and "assistant", and some popular models also use a "system" role.
Other roles 

In [18]:
# Let's see the actual template of this prompt formatter
model.prompt.TEMPLATE

{'user': {'template': '<|startoftranscript|>|source_lang||task||target_lang||pnc|',
  'slots': {'source_lang': nemo.collections.common.prompts.formatter.Text,
   'task': Modality.TextLiteral(allowed_values=('asr', 'ast', 'translate', 'transcribe', 's2t_translation', '<|transcribe|>', '<|translate|>')),
   'target_lang': nemo.collections.common.prompts.formatter.Text,
   'pnc': Modality.TextLiteral(allowed_values=('yes', 'no', 'true', 'True', 'false', 'False', '1', '0', 'pnc', 'nopnc', '<|pnc|>', '<|nopnc|>'))}},
 'assistant': {'template': '|text|<|endoftext|>',
  'slots': {'text': nemo.collections.common.prompts.formatter.Text}}}

---

We see that the template contains two turns - `user` and `assistant`.

User template looks as follows: `<|startoftranscript|>|source_lang||task||target_lang||pnc|`
During execution, we remove the `|` in order to fill in the actual value of the slots provided by the the data loader.

User holds the following allowed slots -
* `source_lang`
* `target_lang`
* `task`
* `pnc`

Similarly, for Assistant template : `|text|<|endoftext|>`

Assistant holds the following allowed slots -
* `text`

### Creating and Using a Custom Prompt Formatter

While we provide a pre-trained model with a pre-defined prompt format, we also enable users to create their own PromptFormatter subclass and change it as needed.

Below, we show a simple modification to the model's PromptFormatter and show how to change it.

In [19]:
# Create a new prompt formatter using the original CanaryPromptFormatter class as baseclass
class CanaryPromptFormatterV2(model.prompt.__class__):

    # make sure to provide a new name
    NAME: str = "canary2"

    # Make any changes as necessary.
    # For this demonstration, we will not change anything other than the name

In [20]:
# Next, lets update the model's prompt formatter
model.change_prompt("canary2")

[NeMo I 2024-11-06 05:36:13 aed_multitask_models:448] Changed prompt format to `canary2`


---

We have now successfully changed the prompt format to `canary2`.

**Note**: It is important to know that when changing the prompt format, the name of the new prompt format class (`canary2` in this case) **has to match** the name of the prompt function registered with `@registered_prompt_format_fn`!

In [21]:
# Check if everything is ok -
model.prompt.__class__.__name__

'CanaryPromptFormatterV2'

In [22]:
model.prompt_format

'canary2'

---
For the rest of the tutorial, we will revert back to the original prompt formatter

In [23]:
model.change_prompt('canary')

[NeMo I 2024-11-06 05:36:13 aed_multitask_models:448] Changed prompt format to `canary`


## Creating / Using a Multi Task Dataset

Now that we have learned how to modify the model's prompt formatter and the underlying format function that maps manifest items into slots to inject into the prompt template, next let's take a look at how to use and create custom datasets for training multi task models.

---

Unlike previous tutorials that showcase how to use pre-defined datasets and point them to your manifest files, we will take a slightly more hands-on approach for multi task modes. This is due to shear flexibility of multi task models - they can do almost any task that you can formulate into a "speech in - text out" problem.

So it is not easy to have a pre-defined dataset class that can handle all new ideas and tasks that researchers can come up with.

Instead, we showcase how to build a custom dataset for yourself and use it with the Multi Task model instead.

---

However, we also provide a base class that can be used as is by users if they dont want the hassle of writing their own datasets.

This is handled by the `PromptedAudioToTextLhotseDataset` -  it maps user defined manifest items to the items defined in the prompt template of the model, so as long as the manifest corresponds to the slots supported by the model, it will be managed by the Dataset automatically.

In [24]:
from nemo.collections.asr.data.audio_to_text_lhotse_prompted import PromptedAudioToTextLhotseDataset

# Uncomment below line to see the class definition of PromptedAudioToTextLhotseDataset
# PromptedAudioToTextLhotseDataset??

### Creating a New Prompted Dataset

In [25]:
import torch.utils.data
from lhotse import CutSet
from lhotse.cut import MixedCut, MonoCut
from lhotse.dataset import AudioSamples
from lhotse.dataset.collation import collate_vectors

from nemo.collections.asr.data.audio_to_text_lhotse_prompted import PromptedAudioToTextLhotseDataset, PromptedAudioToTextMiniBatch

class MyCanaryPromptedAudioToTextLhotseDataset(torch.utils.data.Dataset):
    """
    This dataset is based on :class:`~nemo.collections.asr.data.audio_to_text_lhotse.LhotseSpeechToTextBpeDataset`.
    It is a Lhotse-style dataset that converts a mini-batch of Cuts into tensors.
    The main difference from ``LhotseSpeechToTextBpeDataset`` is that we introduce
    a special prompt format for multitask encoder-decoder models.

    To perform the prompt formatting, we accept a ``prompt_format_fn``.
    It's expected to accept:
    * a ``CutSet`` which it will internally iterate over for utterances, and
    * a ``tokenizer`` object that will be internally used to tokenize the utterances

    Tokenized utterances will be extended with special prompt tokens according to ``prompt_format_fn`` logic.
    We support cuts with multiple supervision segments -- their tokenized texts will be concatenated before we add the prompt tokens.
    This is useful, for example, in code-switched scenarios where each segment is spoken in a different language.
    """

    def __init__(
        self,
        tokenizer: 'TokenizerSpec',
    ):
        super().__init__()
        self.tokenizer = tokenizer
        self.load_audio = AudioSamples(fault_tolerant=True)
        self.padding_value = self.tokenizer.pad_id
        self.prompt_format_fn = get_prompt_format_fn('canary')  # Use the default canary prompt function


    def __getitem__(self, cuts: CutSet) -> PromptedAudioToTextMiniBatch:
        audio, audio_lens, cuts = self.load_audio(cuts)

        prompts_with_answers, prompts, answers = self.prompt_format_fn(cuts, self.tokenizer)
        
        transcript, transcript_lens = self._collate_tokens(answers)
        prompts_with_answers, prompts_with_answers_lens = self._collate_tokens(prompts_with_answers)
        prompts, prompt_lens = self._collate_tokens(prompts)

        return PromptedAudioToTextMiniBatch(
            audio=audio,
            audio_lens=audio_lens,
            transcript=transcript,
            transcript_lens=transcript_lens,
            prompt=prompts,
            prompt_lens=prompt_lens,
            prompted_transcript=prompts_with_answers,
            prompted_transcript_lens=prompts_with_answers_lens,
            cuts=cuts.drop_in_memory_data(),
        )

    def _collate_tokens(self, tokens: list[list[int] | torch.Tensor]) -> tuple[torch.Tensor, torch.Tensor]:
        tokens = [torch.as_tensor(t) for t in tokens]
        token_lens = torch.tensor([t.size(0) for t in tokens], dtype=torch.long)
        tokens = collate_vectors(tokens, padding_value=self.padding_value)
        return tokens, token_lens


---

The above class is mostly a demonstration, but it showcases how users might flexibly change the prompt formatter, prompt format function and even the data set that handles these two in a flexible way.

The order of operations is usually this -

1) Create a new Prompt Formatter class - this denotes the slots that each turn can have (including new task inputs or other values). This class is auto registered.
2) Create a new Prompt Format function - Using `@registered_prompt_format_fn` decorator, write a custom function that accepts args and processes the provided input data from a manifest.
3) Create a new Dataset class (usually based on the `PromptedAudioToTextLhotseDataset` dataset) that uses the Prompt Format function to convert manifest items into nicely formatted samples that can be passed to the Prompt Formatter.

# Preparing a Canary Dataset

Now that we have all the pieces together on the model side, let's take a look on the data side.

## Required Roles Defined by Prompt Format

These are the available 'roles' available in the prompt format - they denote at each turn, one role can be enabled and its input or output can be calculated.

In [26]:
model.prompt.get_roles()

['user', 'assistant']

In [27]:
for role in model.prompt.get_roles():
    print(role, model.prompt.get_slots(role))
    print()

user {'source_lang': <class 'nemo.collections.common.prompts.formatter.Text'>, 'task': Modality.TextLiteral(allowed_values=('asr', 'ast', 'translate', 'transcribe', 's2t_translation', '<|transcribe|>', '<|translate|>')), 'target_lang': <class 'nemo.collections.common.prompts.formatter.Text'>, 'pnc': Modality.TextLiteral(allowed_values=('yes', 'no', 'true', 'True', 'false', 'False', '1', '0', 'pnc', 'nopnc', '<|pnc|>', '<|nopnc|>'))}

assistant {'text': <class 'nemo.collections.common.prompts.formatter.Text'>}



## Create a Data Module

Data Modules are one way of organizing datasets in PyTorch Lightning. It provides a unified place where data loading and processing can be potentially handled.

**Note**: This isn't strictly necessary - you can achieve the same using just Pytorch dataloaders directly and passing it to Trainer.fit() but we showcase a data module codebase that can be extended by the user.

----

In our CanaryAN4DataModule - we will perform two tasks. One is En ASR - transcribing the AN4 English dataset. Another is En to De AST - directly translating the english audio to German text.

For simplicity's sake, we will use a small off-the-shelf model to perform the translation of English Transcripts to German.

---

In NeMo 2.0, we utilize [Lhotse](https://github.com/lhotse-speech/lhotse) as our data backbone for speech tasks, which simplifies using custom speech datasets.

Most of the magic is handled by the following code

```python
from nemo.collections.common.data.lhotse import get_lhotse_dataloader_from_config

get_lhotse_dataloader_from_config(
    OmegaConf.create(config),  # Pass in a config that points to the manifest files and other arguments
    global_rank=self.trainer.global_rank,
    world_size=self.trainer.world_size,
    # Pass in the dataset class for Lhotse to handle. This class now receives CutSet as input.
    dataset=MyCanaryPromptedAudioToTextLhotseDataset(tokenizer=self.tokenizer),
)
```

In [28]:
import os
import glob
import json
import copy
import subprocess
import tarfile
import wget
import librosa
import tqdm
from omegaconf import OmegaConf

from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as L

from transformers import T5Tokenizer, T5ForConditionalGeneration

from nemo.collections.asr.parts.utils.manifest_utils import read_manifest, write_manifest
from nemo.collections.common.data.lhotse import get_lhotse_dataloader_from_config


# Function to build a manifest
def build_manifest(transcripts_path, manifest_path, wav_path, data_dir):
    with open(transcripts_path, 'r') as fin:
        with open(manifest_path, 'w') as fout:
            for line in fin:
                # Lines look like this:
                # <s> transcript </s> (fileID)
                transcript = line[: line.find('(')-1].lower()
                transcript = transcript.replace('<s>', '').replace('</s>', '')
                transcript = transcript.strip()

                file_id = line[line.find('(')+1 : -2]  # e.g. "cen4-fash-b"
                audio_path = os.path.join(
                    data_dir, wav_path,
                    file_id[file_id.find('-')+1 : file_id.rfind('-')],
                    file_id + '.wav')

                duration = librosa.core.get_duration(path=audio_path)

                # Write the metadata to the manifest
                metadata = {
                    "audio_filepath": audio_path,
                    "duration": duration,
                    "text": transcript,
                    "pnc": "no",
                    "source_lang": "en",
                    "target_lang": "en",
                    "task": "asr",
                }
                json.dump(metadata, fout)
                fout.write('\n')

    return manifest_path


class CanaryAN4DataModule(L.LightningDataModule):

    def __init__(self, tokenizer, data_dir: str = "./an4/", batch_size=8):
        super().__init__()
        self.tokenizer = tokenizer
        self.data_dir = data_dir
        self.batch_size = batch_size

        # ASR manifests
        self.train_manifest = data_dir + '/an4/train_manifest.json'
        self.test_manifest = data_dir + '/an4/test_manifest.json'

        # AST manifests
        self.ast_train_manifest = data_dir + '/an4/ast_train_manifest.json'
        self.ast_test_manifest = data_dir + '/an4/ast_test_manifest.json'

        # Combined manifests
        self.combined_train_manifest = data_dir + '/an4/combined_train_manifest.json'
        self.combined_test_manifest = data_dir + '/an4/combined_test_manifest.json'

    def setup(self, stage):
        # make assignments here (val/train/test split)
        # called on every process in DDP
        # Assign train/val datasets for use in dataloaders
        pass

    def train_dataloader(self):
        config = {'manifest_filepath': self.combined_train_manifest, 'batch_size': self.batch_size,
                  'num_workers': 4, 'shuffle': True, 'min_duration': 0.3, 'max_duration': 10.0}
        return self._setup_dataloader(config)

    def val_dataloader(self):
        config = {'manifest_filepath': self.combined_test_manifest, 'batch_size': self.batch_size,
                  'num_workers': 4, 'shuffle': False, 'min_duration': 0.3, 'max_duration': 10.0}
        return self._setup_dataloader(config)

    def test_dataloader(self):
        config = {'manifest_filepath': self.combined_test_manifest, 'batch_size': self.batch_size,
                  'num_workers': 4, 'shuffle': False, 'min_duration': 0.3, 'max_duration': 10.0}
        return self._setup_dataloader(config)

    def teardown(self, stage):
        # clean up after fit or test
        # called on every process in DDP
        pass

    def _setup_dataloader(self, config):
        """
        The main function that creates the data loader using Lhotse's integration with NeMo.
        """
        return get_lhotse_dataloader_from_config(
                OmegaConf.create(config),
                global_rank=self.trainer.global_rank,
                world_size=self.trainer.world_size,
                # Note the passing of our custom dataset
                dataset=MyCanaryPromptedAudioToTextLhotseDataset(tokenizer=self.tokenizer),
            )

    def prepare_data(self):
        # download, split, etc...
        # only called on 1 GPU/TPU in distributed
        if not os.path.exists(self.data_dir):
            os.makedirs(self.data_dir)

        data_dir = self.data_dir
        if not os.path.exists(data_dir + '/an4_sphere.tar.gz'):
            an4_url = 'https://dldata-public.s3.us-east-2.amazonaws.com/an4_sphere.tar.gz'
            an4_path = wget.download(an4_url, data_dir)
            print(f"Dataset downloaded at: {an4_path}")
        else:
            print("Tarfile already exists.")
            an4_path = data_dir + '/an4_sphere.tar.gz'

        if not os.path.exists(data_dir + '/an4/'):
            # Untar and convert .sph to .wav (using sox)
            tar = tarfile.open(an4_path)
            tar.extractall(path=data_dir)

            print("Converting .sph to .wav...")
            sph_list = glob.glob(data_dir + '/an4/**/*.sph', recursive=True)
            for sph_path in sph_list:
                wav_path = sph_path[:-4] + '.wav'
                cmd = ["sox", sph_path, wav_path]
                subprocess.run(cmd)
        print("Finished conversion.\n******")

        # Building Manifests
        print("******")
        train_transcripts = data_dir + '/an4/etc/an4_train.transcription'
        train_manifest = self.train_manifest
        if not os.path.isfile(train_manifest):
            build_manifest(train_transcripts, train_manifest, 'an4/wav/an4_clstk', data_dir)
            print("Training manifest created.")

        test_transcripts = data_dir + '/an4/etc/an4_test.transcription'
        test_manifest = self.test_manifest
        if not os.path.isfile(test_manifest):
            build_manifest(test_transcripts, test_manifest, 'an4/wav/an4test_clstk', data_dir)
            print("Test manifest created.")
        print("*** Wrote manifests for Eng ***")

        train_manifest_data = read_manifest(self.train_manifest)
        test_manifest_data = read_manifest(self.test_manifest)

        if not os.path.isfile(self.ast_train_manifest) or not os.path.isfile(self.ast_test_manifest) or not os.path.isfile(self.combined_train_manifest) or not os.path.isfile(self.combined_test_manifest):
            tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
            t5_model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

            if torch.cuda.is_available():
                t5_model = t5_model.cuda()

            def pipe(text):
                if isinstance(text, str):
                    text = [text]

                prefix = "translate English to German"
                prompts = [prefix + ": " + x for x in text]
                input_ids = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).input_ids
                input_ids = input_ids.to(t5_model.device)
                outputs = t5_model.generate(input_ids, max_new_tokens=64)
                return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

            ast_train_manifest_data = copy.deepcopy(train_manifest_data)
            ast_test_manifest_data = copy.deepcopy(test_manifest_data)

            print("Translating train set")
            train_texts = [x['text'] for x in train_manifest_data]
            BATCH_SIZE = 32

            for i in tqdm.tqdm(range(0, len(train_texts), BATCH_SIZE), total=len(train_texts) // BATCH_SIZE):
                batch_texts = train_texts[i:i+BATCH_SIZE]
                batch_texts = pipe(batch_texts)
                for j, text in enumerate(batch_texts):
                    ast_train_manifest_data[i+j]['text'] = text
                    ast_train_manifest_data[i+j]['task'] = 'ast'
                    ast_train_manifest_data[i+j]['target_lang'] = 'de'

            print("Translating test set")
            for data in tqdm.tqdm(ast_test_manifest_data, total=len(ast_test_manifest_data)):
                data['text'] = pipe(data['text'])[0]
                data['task'] = 'ast'
                data['target_lang'] = 'de'

            write_manifest(self.ast_train_manifest, ast_train_manifest_data)
            write_manifest(self.ast_test_manifest, ast_test_manifest_data)

            print("*** Wrote ast manifests ***")

            combined_train, combined_test = [], []
            combined_train.extend(train_manifest_data)
            combined_train.extend(ast_train_manifest_data)

            combined_test.extend(test_manifest_data)
            combined_test.extend(ast_test_manifest_data)

            write_manifest(self.combined_train_manifest, combined_train)
            write_manifest(self.combined_test_manifest, combined_test)
            print("*** Wrote combined manifests ***")

        else:
            print("*** Wrote ast and combined manifests ***")


---

Each item in the prepared manifest has the following items by default.

As you will recognize, these are the same keys provided by the `CanaryPromptFormatter` classes `slots` argument, so each of these values in the is mapped back to those slots.

```python
metadata = {
    "audio_filepath": audio_path,
    "duration": duration,
    "text": transcript,
    "pnc": "no",
    "source_lang": "en",
    "target_lang": "en",
    "task": "asr",
}
```

The most important function in the Data Module above is `prepare_data()`:

1) It first downloads and converts the AN4 audio files to wav files.
2) Then it writes a new manifest file with the above keys for ASR task
3) It then translates the En transcripts with a `t5-small` model to generate German transcripts
4) Finally it writes another manifest for the AST task with these translated texts.
5) Finally it builds a combined manifest item for both ASR (en) and AST (en to de) multi-task training

**Note**: We are using prepare_data() only for demonstration. Normally, users should process before experimentation, and so they would only need to implement methods above prepare_data() in their Data Module.

## Download and Prepare Dataset

In [29]:
data_module = CanaryAN4DataModule(tokenizer=model.tokenizer, batch_size=16)

In [30]:
data_module.prepare_data()

Tarfile already exists.
Finished conversion.
******
******
*** Wrote manifests for Eng ***
*** Wrote ast and combined manifests ***


In [31]:
!head -n 5 {data_module.train_manifest}

{"audio_filepath": "./an4/an4/wav/an4_clstk/fash/an251-fash-b.wav", "duration": 1.0, "text": "yes", "pnc": "no", "source_lang": "en", "target_lang": "en", "task": "asr"}
{"audio_filepath": "./an4/an4/wav/an4_clstk/fash/an253-fash-b.wav", "duration": 0.7, "text": "go", "pnc": "no", "source_lang": "en", "target_lang": "en", "task": "asr"}
{"audio_filepath": "./an4/an4/wav/an4_clstk/fash/an254-fash-b.wav", "duration": 0.9, "text": "yes", "pnc": "no", "source_lang": "en", "target_lang": "en", "task": "asr"}
{"audio_filepath": "./an4/an4/wav/an4_clstk/fash/an255-fash-b.wav", "duration": 2.6, "text": "u m n y h six", "pnc": "no", "source_lang": "en", "target_lang": "en", "task": "asr"}
{"audio_filepath": "./an4/an4/wav/an4_clstk/fash/cen1-fash-b.wav", "duration": 3.5, "text": "h i n i c h", "pnc": "no", "source_lang": "en", "target_lang": "en", "task": "asr"}


In [32]:
!head -n 5 {data_module.ast_train_manifest}

{"audio_filepath": "./an4/an4/wav/an4_clstk/fash/an251-fash-b.wav", "duration": 1.0, "text": "ja", "pnc": "no", "source_lang": "en", "target_lang": "de", "task": "ast"}
{"audio_filepath": "./an4/an4/wav/an4_clstk/fash/an253-fash-b.wav", "duration": 0.7, "text": "go", "pnc": "no", "source_lang": "en", "target_lang": "de", "task": "ast"}
{"audio_filepath": "./an4/an4/wav/an4_clstk/fash/an254-fash-b.wav", "duration": 0.9, "text": "ja", "pnc": "no", "source_lang": "en", "target_lang": "de", "task": "ast"}
{"audio_filepath": "./an4/an4/wav/an4_clstk/fash/an255-fash-b.wav", "duration": 2.6, "text": "u m n y h six", "pnc": "no", "source_lang": "en", "target_lang": "de", "task": "ast"}
{"audio_filepath": "./an4/an4/wav/an4_clstk/fash/cen1-fash-b.wav", "duration": 3.5, "text": "h i n i c h", "pnc": "no", "source_lang": "en", "target_lang": "de", "task": "ast"}


# Evaluate Model before Training

Canary Multi Task model is already very capable, achieving strong scores on multiple benchmarks. So we first evaluate the baseline numbers on the two tasks

1) ASR: WER calculation on transcripts

2) AST: SacreBLEU calculation on translations

In [33]:
from nemo.collections.asr.metrics.wer import word_error_rate
from torchmetrics.text import SacreBLEUScore

In [34]:
asr_test = read_manifest(data_module.test_manifest)
ast_test = read_manifest(data_module.ast_test_manifest)

In [35]:
asr_filepaths = [x['audio_filepath'] for x in asr_test]
asr_gt = [x['text'] for x in asr_test]

ast_filepaths = [x['audio_filepath'] for x in ast_test]
ast_gt = [x['text'] for x in ast_test]

print("Num files:", len(asr_filepaths))

Num files: 130


In [36]:
if torch.cuda.is_available():
    model = model.cuda()  # move model to gpu
    model = model.to(torch.bfloat16)  # cast full model to bfloat16

In [37]:
asr_preds = model.transcribe(asr_filepaths, pnc='no', task='asr', source_lang='en', target_lang='en', batch_size=32)

[NeMo W 2024-11-06 05:36:14 dataloader:206] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process, possibly impacting the training speed if your tokenizer is very large. If the impact is noticable, set pretokenize=False in dataloader config. (note: that will disable token-per-second filtering and 2D bucketing features)
Transcribing: 0it [00:00, ?it/s][NeMo W 2024-11-06 05:36:16 attention_adapter_mixin:125] No adapter compatible with the current module. Skipping adapter forward pass.
[NeMo W 2024-11-06 05:36:16 deprecated:65] Function ``_transcribe_output_processing`` is deprecated. The return type of args will be updated in the upcoming release to ensure a consistent output format across all decoder types, such that a Hypothesis object is always returned.
Transcribing: 5it [00:05,  1.03s/it]


In [38]:
ast_preds = model.transcribe(ast_filepaths, pnc='no', task='ast', source_lang='en', target_lang='de', batch_size=32)

[NeMo W 2024-11-06 05:36:19 dataloader:206] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process, possibly impacting the training speed if your tokenizer is very large. If the impact is noticable, set pretokenize=False in dataloader config. (note: that will disable token-per-second filtering and 2D bucketing features)
Transcribing: 5it [00:04,  1.14it/s]


In [39]:
wer = word_error_rate(asr_preds, asr_gt)
print("WER", wer)

sacrebleu = SacreBLEUScore(n_gram=4)
scores = []
preds = []
gts = []
for pred, gt in zip(ast_preds, ast_gt):
    preds.append(pred)
    gts.append([gt])

# bleu = sum(scores) / len(scores)
sacrebleu.update(preds, gts)
bleu = sacrebleu.compute()
print("BLEU", bleu.item() * 100)

WER 0.040103492884864166
BLEU 66.23761057853699


# Train Model

Finally, now that adapters have been prepared, model has been evaluated for a baseline and the dataset is prepared, it's time to train the adapter weights on the new datasets.

---

First, we update the optimizer and scheduler config

In [40]:
print(OmegaConf.to_yaml(model.cfg.optim))

name: adamw
lr: 0.0003
betas:
- 0.9
- 0.98
weight_decay: 0.001
sched:
  name: InverseSquareRootAnnealing
  warmup_steps: 2500
  warmup_ratio: null
  min_lr: 1.0e-06



In [41]:
# Setup optimization
model.cfg.optim.lr = 3e-4
model.cfg.optim.sched.warmup_steps = 25

---

Next, we setup a Lightning Trainer and Experiment Manager

In [42]:
from omegaconf import OmegaConf
from nemo.utils import exp_manager

In [43]:
trainer = L.Trainer(max_steps=200, accumulate_grad_batches=1, logger=False, enable_checkpointing=False, check_val_every_n_epoch=5)

Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [44]:
# # Environment variable generally used for multi-node multi-gpu training.
# # In notebook environments, this flag is unnecessary and can cause logs of multiple training runs to overwrite each other.
# os.environ.pop('NEMO_EXPM_VERSION', None)

# config = exp_manager.ExpManagerConfig(
#     exp_dir=f'experiments/canary/',
#     name=f"Canary-Model-Adapter-Training",
#     checkpoint_callback_params=exp_manager.CallbackParams(
#         monitor="val_wer",
#         mode="min",
#         always_save_nemo=False,
#         save_best_model=False,
#     ),
# )

# config = OmegaConf.structured(config)

# logdir = exp_manager.exp_manager(trainer, config)

---

Begin training !

In [45]:
trainer.fit(model, data_module)

You are using a CUDA device ('NVIDIA RTX 6000 Ada Generation') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Tarfile already exists.
Finished conversion.
******
******
*** Wrote manifests for Eng ***


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


*** Wrote ast and combined manifests ***
[NeMo I 2024-11-06 05:36:24 modelPT:787] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.0003
        maximize: False
        weight_decay: 0.001
    )
[NeMo I 2024-11-06 05:36:24 lr_scheduler:948] Scheduler "<nemo.core.optim.lr_scheduler.InverseSquareRootAnnealing object at 0x7f8a419ae2f0>" 
    will be used during training (effective maximum steps = 200) - 
    Parameters : 
    (warmup_steps: 25
    warmup_ratio: null
    min_lr: 1.0e-06
    max_steps: 200
    )



  | Name                 | Type                              | Params | Mode 
-----------------------------------------------------------------------------------
0 | preprocessor         | AudioToMelSpectrogramPreprocessor | 0      | eval 
1 | encoder              | ConformerEncoderAdapter           | 609 M  | train
2 | encoder_decoder_proj | Identity                          | 0      | eval 
3 | transf_decoder       | TransformerDecoderNMAdapter       | 407 M  | train
4 | log_softmax          | TokenClassifier                   | 4.2 M  | eval 
5 | loss                 | SmoothedCrossEntropyLoss          | 0      | eval 
6 | spec_augmentation    | SpectrogramAugmentation           | 0      | eval 
7 | val_loss             | GlobalAverageLossMetric           | 0      | eval 
8 | wer                  | WER                               | 0      | eval 
9 | bleu                 | BLEU                              | 0      | eval 
---------------------------------------------------------

Sanity Checking: |                                                                                            …

[NeMo I 2024-11-06 05:36:24 dataloader:178] We will be using a Lhotse DataLoader.
[NeMo I 2024-11-06 05:36:24 dataloader:327] Creating a Lhotse DynamicCutSampler (bucketing is disabled, (max_batch_duration=None max_batch_size=16)
[NeMo I 2024-11-06 05:36:25 wer:329] 
    
[NeMo I 2024-11-06 05:36:25 wer:330] reference: rubout g m e f three nine
[NeMo I 2024-11-06 05:36:25 wer:331] predicted: rub out g m e f three nine
[NeMo I 2024-11-06 05:36:26 wer:329] 
    
[NeMo I 2024-11-06 05:36:26 wer:330] reference: j p e g four
[NeMo I 2024-11-06 05:36:26 wer:331] predicted: j p e g four
[NeMo I 2024-11-06 05:36:26 dataloader:178] We will be using a Lhotse DataLoader.
[NeMo I 2024-11-06 05:36:26 dataloader:327] Creating a Lhotse DynamicCutSampler (bucketing is disabled, (max_batch_duration=None max_batch_size=16)


Training: |                                                                                                   …

[NeMo W 2024-11-06 05:36:27 nemo_logging:361] /home/pzelasko/miniconda3/envs/nemo2/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:212: You called `self.log('num_frames', ...)` in your `training_step` but the value needs to be floating to be reduced. Converting it to torch.float32. You can silence this warning by converting the value to floating point yourself. If you don't intend to reduce the value (for instance when logging the global step or epoch) then you can use `self.logger.log_metrics({'num_frames': ...})` instead.
    
[NeMo W 2024-11-06 05:36:27 nemo_logging:361] /home/pzelasko/miniconda3/envs/nemo2/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:212: You called `self.log('num_tokens', ...)` in your `training_step` but the value needs to be floating to be reduced. Converting it to torch.float32. You can silence this warning by converting the value to floating point yourself. If you don'

---

Save just the adapter parameters - which is less than 2 MB !

In [46]:
model.save_adapters("adapters.pt")
!ls -l -- *.pt
!du -sh *.pt

-rw-rw-r-- 1 pzelasko pzelasko 1870390 Nov  6 05:37 adapters.pt
1.8M	adapters.pt


# Evaluate after Adaptation

Now that the model is done training, lets evaluate its scores on the test set again.
We should see a markedly higher translation BLEU and lower WER from above.

In [47]:
asr_test = read_manifest(data_module.test_manifest)
ast_test = read_manifest(data_module.ast_test_manifest)

In [48]:
asr_filepaths = [x['audio_filepath'] for x in asr_test]
asr_gt = [x['text'] for x in asr_test]

ast_filepaths = [x['audio_filepath'] for x in ast_test]
ast_gt = [x['text'] for x in ast_test]

print("Num files:", len(asr_filepaths))

Num files: 130


In [49]:
if torch.cuda.is_available():
    model = model.cuda()
    model = model.to(torch.bfloat16)

In [50]:
asr_preds = model.transcribe(asr_filepaths, pnc='no', task='asr', source_lang='en', target_lang='en', batch_size=32)

[NeMo W 2024-11-06 05:37:05 dataloader:206] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process, possibly impacting the training speed if your tokenizer is very large. If the impact is noticable, set pretokenize=False in dataloader config. (note: that will disable token-per-second filtering and 2D bucketing features)
Transcribing: 5it [00:04,  1.15it/s]


In [51]:
ast_preds = model.transcribe(ast_filepaths, pnc='no', task='ast', source_lang='en', target_lang='de', batch_size=32)

[NeMo W 2024-11-06 05:37:10 dataloader:206] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process, possibly impacting the training speed if your tokenizer is very large. If the impact is noticable, set pretokenize=False in dataloader config. (note: that will disable token-per-second filtering and 2D bucketing features)
Transcribing: 5it [00:04,  1.10it/s]


In [52]:
from nemo.collections.asr.metrics.wer import word_error_rate
from torchmetrics.text import SacreBLEUScore

In [53]:
wer = word_error_rate(asr_preds, asr_gt)
print("WER", wer)

WER 0.02069857697283312


In [54]:
sacrebleu = SacreBLEUScore(n_gram=4)
scores = []
preds = []
gts = []
for pred, gt in zip(ast_preds, ast_gt):
    preds.append(pred)
    gts.append([gt])

# bleu = sum(scores) / len(scores)
sacrebleu.update(preds, gts)
bleu = sacrebleu.compute()
print("BLEU", bleu.item() * 100)

BLEU 72.54462242126465


# Conclusion

In this tutorial we added adapters to a Multi Task model (Nvidia Canary) and show how to create a custom dataset to finetune a canary model to a new dataset with previous tasks such as ASR and AST. The primary goal of this tutorial was to show how to flexibly adapt a Canary model to any of the pre-existing tasks.

In a future tutorial, we will show how to add additional tasks to a pre-trained Canary, so that you can leverage the pre-trained encoder and decoder for your own custom tasks!